In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
from src.data_loader import read_data

train_data = read_data('train')
val_data = read_data('val')
test_data = read_data('test')

100%|██████████| 200/200 [00:00<00:00, 2301.82it/s]


In [3]:
from src.config import BATCH_SIZE
import random 
import numpy as np 
import torch 

from src.networks.CLIP_Basic import CLIP_only_dataset

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
g = torch.Generator()
g.manual_seed(0)

def get_dataloader(data):
    dataset = CLIP_only_dataset(data)
    data_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE,shuffle=True, worker_init_fn=seed_worker, generator=g)
    return data_loader
    
train_loader = get_dataloader(train_data)
val_loader = get_dataloader(val_data)
test_loader = get_dataloader(test_data)

In [6]:
from src.trainer import Trainer
from src.networks.CLIP_Basic import CLIP_only_Model, CLIP_only_Input_transformer
import torch.nn as nn
import torch 

trainer = Trainer(10)

#Set data
trainer.set_data(train_loader, val_loader)

#set model 
model = CLIP_only_Model().cuda()
input_transformer = CLIP_only_Input_transformer()
trainer.set_model(model, input_transformer)

#Backpropagation
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
trainer.set_optimizer(optimizer, loss_fn)
trainer.train()

100%|██████████| 2/2 [00:00<00:00,  2.92it/s]




For epoch = 0
Training Loss = 0.5415075759996067 | Training Accuracy = 0.05274621212121212
Validation Loss = 0.38505013287067413|Validation Accuracy = 0.09576612903225806




100%|██████████| 2/2 [00:00<00:00,  2.90it/s]




For epoch = 1
Training Loss = 0.30615095726468344 | Training Accuracy = 0.1393939393939394
Validation Loss = 0.3033136576414108|Validation Accuracy = 0.078125




100%|██████████| 2/2 [00:00<00:00,  2.90it/s]




For epoch = 2
Training Loss = 0.2624136582016945 | Training Accuracy = 0.17263257575757576
Validation Loss = 0.34145933389663696|Validation Accuracy = 0.11139112903225806




 73%|███████▎  | 16/22 [00:05<00:02,  2.70it/s]


KeyboardInterrupt: 

In [7]:
trainer.test(test_loader)

100%|██████████| 22/22 [00:07<00:00,  2.88it/s]


Accuracy Score = 0.1703056768558952
Macro F1 score Score = 0.10090839107164283
Micro F1 score Score = 0.5744751749416861
Classification Report


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.89      0.74      0.81       434
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0
          11       0.00      0.00      0.00         0
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00 

C:\Users\ADITYA RATHORE\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ADITYA RATHORE\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ADITYA RATHORE\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(